# 创建并运行第一个图

In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y +2

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [3]:
result

42

In [4]:
sess.close()

In [5]:
result

42

In [6]:
x

<tf.Variable 'x:0' shape=() dtype=int32_ref>

In [7]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [8]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [10]:
result

42

In [11]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()

In [12]:
result

42

In [13]:
sess.close()

# 管理图

In [14]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [16]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [17]:
x2.graph is graph

True

In [18]:
x2.graph is tf.get_default_graph()

False

# 节点的生命周期

In [20]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [21]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


# 线性回归

In [22]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [23]:
theta_value

array([[-3.7171074e+01],
       [ 4.3633682e-01],
       [ 9.3871783e-03],
       [-1.0717344e-01],
       [ 6.4540231e-01],
       [-4.1238391e-06],
       [-3.7809242e-03],
       [-4.2373490e-01],
       [-4.3720812e-01]], dtype=float32)

## 梯度下降法求解

In [29]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [39]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE= 12.408014
Epoch 100 MSE= 0.75519687
Epoch 200 MSE= 0.5420874
Epoch 300 MSE= 0.5331699
Epoch 400 MSE= 0.53053814
Epoch 500 MSE= 0.5287963
Epoch 600 MSE= 0.527549
Epoch 700 MSE= 0.52664965
Epoch 800 MSE= 0.5260011
Epoch 900 MSE= 0.52553326


In [35]:
best_theta

array([[ 2.0685523e+00],
       [ 8.1063598e-01],
       [ 1.2685776e-01],
       [-2.0784086e-01],
       [ 2.4839850e-01],
       [-1.3083885e-03],
       [-3.9607048e-02],
       [-8.5861266e-01],
       [-8.2600272e-01]], dtype=float32)

## 自动微分

In [41]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0] # 自动微分
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE= 12.408014
Epoch 100 MSE= 0.75519687
Epoch 200 MSE= 0.5420874
Epoch 300 MSE= 0.53317
Epoch 400 MSE= 0.53053814
Epoch 500 MSE= 0.5287964
Epoch 600 MSE= 0.527549
Epoch 700 MSE= 0.5266497
Epoch 800 MSE= 0.5260011
Epoch 900 MSE= 0.5255332


In [42]:
best_theta

array([[ 2.0685525e+00],
       [ 8.1063598e-01],
       [ 1.2685777e-01],
       [-2.0784083e-01],
       [ 2.4839847e-01],
       [-1.3083883e-03],
       [-3.9607048e-02],
       [-8.5861266e-01],
       [-8.2600272e-01]], dtype=float32)

## 使用优化器

In [43]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE= 12.408014
Epoch 100 MSE= 0.75519687
Epoch 200 MSE= 0.5420874
Epoch 300 MSE= 0.53317
Epoch 400 MSE= 0.53053814
Epoch 500 MSE= 0.5287964
Epoch 600 MSE= 0.527549
Epoch 700 MSE= 0.5266497
Epoch 800 MSE= 0.5260011
Epoch 900 MSE= 0.5255332


In [44]:
best_theta

array([[ 2.0685525e+00],
       [ 8.1063598e-01],
       [ 1.2685777e-01],
       [-2.0784083e-01],
       [ 2.4839847e-01],
       [-1.3083883e-03],
       [-3.9607048e-02],
       [-8.5861266e-01],
       [-8.2600272e-01]], dtype=float32)

# 向算法提供数据

In [46]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [47]:
B_val_1

array([[6., 7., 8.]], dtype=float32)

In [48]:
B_val_2

array([[ 9., 10., 11.],
       [12., 13., 14.]], dtype=float32)

In [59]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch


In [60]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
learning_rate = 0.01
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()

In [61]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

# 保存和载入模型

In [63]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')

## 载入模型

In [65]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


# TensorBoard 可视化图和训练曲线

In [84]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)

In [86]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
learning_rate = 0.01
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()

# 命名范围

In [87]:
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')

In [88]:
error.op.name

'loss/sub'

In [89]:
mse.op.name

'loss/mse'

# 模块化

In [90]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")
output = tf.add(relu1, relu2, name="output")

In [95]:
def relu(X):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, 0., name='relu')

In [96]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

# 共享变量

In [99]:
def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, threshold, name='relu')

In [101]:
n_features = 3
threshold = tf.Variable(0.0, name='threshold')
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name='output')